### **Intro and Acknowledgements**
Author: Saanvi Molugu

This Jupyter Notebook describes the preprocessing of the data collected from the ChEMBL database, and pairing drug activity data with molecular fingerprints of various potential therapeutic agents.

The target of interest is **ERK2**, a major cell signalling protein involved in multiple forms of [cancers](https://en.wikipedia.org/wiki/MAPK1).

The molecular fingerprint database, called PaDEL, has been generously offered by Dr. Yap's lab: https://doi.org/10.1002/jcc.21707

Inspiration and guidance for this project was found through [Chanin Nantasenamat's](https://https://www.youtube.com/@DataProfessor) Bioinformatics Series.

### Retrieving ChEMBL Data

(day 1 stuff)


In [2]:
# generic imports and mounting colab to drive for easy file access
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# enables access to ChEMBL's API to retrieve chemical data about ERK2
! pip install chembl_webresource_client
from chembl_webresource_client.new_client import new_client

# Target search for coronavirus
target = new_client.target
target_query = target.search('ERK2')
targets = pd.DataFrame.from_dict(target_query)
# filtering for Homo sapiens (human)
human_target = targets[targets['organism'] == "Homo sapiens"]
# selecting best human target
selected_target = human_target.target_chembl_id[1]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.4 MB/s eta 0:00:00


Bioactivity is commonly recorded by **IC50**, or the dose of the substrate needed to target 50% of all molecules. A smaller *IC50* value indicates a lower dose required, or higher potency, and implies greater reactivity with the drug to the target protein.

To simplify data, this project focuses on this aspect of bioactivity and as such the ChEMBL data is filtered for activity type of IC50.

In [4]:
activity = new_client.activity
bioact = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")
bioact_df = pd.DataFrame.from_dict(bioact)

#saving data
bioact_df.to_csv("ERK2_chembl_preprocessed.csv")

# testing df by printing last 3 rows
# bioact_df.tail(3)
# ensuring that bioactivity data only includes IC50 measurements
# bioact_df.standard_type.unique()

In [5]:
# making directories in google drive and copying data there
! mkdir "/content/drive/My Drive/ERK2 Project"
! mkdir "/content/drive/My Drive/ERK2 Project/data"
! cp ERK2_chembl_preprocessed.csv "/content/drive/My Drive/ERK2 Project/data"
! ls -l "/content/drive/My Drive/ERK2 Project/data"

mkdir: cannot create directory ‘/content/drive/My Drive/ERK2 Project’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/ERK2 Project/data’: File exists
total 33774
-rw------- 1 root root  4964044 Aug 16 05:59 ERK2_chembl_preprocessed.csv
-rw------- 1 root root   357842 Aug  8 15:12 ERK2_chembl_processed.csv
-rw------- 1 root root  6871154 Aug  8 16:05 ERK2_pIC50_fp_train.csv
-rw------- 1 root root   273776 Aug  8 03:38 ERK2.smi
-rw------- 1 root root    55461 Aug 16 05:20 experimental_predicted_pIC50.png
-rw------- 1 root root 21787005 Aug  9 16:46 model.pkl
-rw------- 1 root root   273746 Aug  8 15:18 molecule.smi


###Preprocessing ERK2 Data

In [6]:
# upload preprocessed data from google drive
! cd "/content/drive/My Drive/ERK2 Project/data/"
file_path = "/content/drive/My Drive/ERK2 Project/data/ERK2_chembl_preprocessed.csv"
bioact_df = pd.read_csv(file_path)
bioact_df[['molecule_chembl_id', 'standard_value']].head()  # printing first 5 rows of data

,molecule_chembl_id,standard_value
0,CHEMBL113851,NaN
1,CHEMBL263536,NaN
2,CHEMBL397591,2500.0
3,CHEMBL35482,2800.0
4,CHEMBL388978,2.5


In [21]:
# dose in units of only nM (nanomolar) will be used, so other IC50 units will be filtered
bioact_filtered = bioact_df[bioact_df['standard_units'] == "nM"]


# filter any rows with no IC50 information or SMILES information
bioact_filtered = bioact_filtered[bioact_filtered.standard_value.notna()]
bioact_filtered = bioact_filtered[bioact_filtered.canonical_smiles.notna()]

bioact_filtered['standard_value'].isna().sum()

# the standard_value will now be renamed to pIC50 for ease of calculations for predictive model (discussed below)

bioact_filtered.rename(columns={'standard_value': 'pIC50'}, inplace=True)

# We can see the highest IC50 value in our dataset here,
# which doesn't cause issues when we switch to pIC50!
bioact_filtered[['pIC50']].describe()

,pIC50
count,3846.000000
mean,5245.898889
std,24554.917658
min,0.004310
25%,1.537750
50%,9.813000
75%,242.675000
max,850000.000000


In [22]:
# converting the IC50 column to pIC50 for ease of calculations for predictive model
# why? pIC50 reduces the skew caused by large IC50 entries, preventing outliers from
# altering the accuracy of our model.
# the model requires a pIC50 that is positive for the input, so all existing
# IC50 concentrations (nM) will be capped at 100,000,000nM (or 1 pIC50)

bioact_filtered['pIC50'] = bioact_filtered['pIC50'].apply(lambda x: min(x, 100000000))
bioact_filtered['pIC50'] = bioact_filtered['pIC50'].apply(lambda x: -np.log10(x*(10**-9)))
# bioact_filtered[['molecule_chembl_id','pIC50']].head()

selection = ['canonical_smiles', 'molecule_chembl_id', 'pIC50']
bioact_filtered = bioact_filtered[selection]

In [23]:
#renaming columns and saving to google drive
bioact_filtered.rename(columns={'canonical_smiles':'smiles', 'molecule_chembl_id':'chembl_id'}, inplace = True)
bioact_filtered.head()
# saving processed ChEMBL data

bioact_filtered.to_csv("ERK2_chembl_processed.csv")
! cp ERK2_chembl_processed.csv "/content/drive/My Drive/ERK2 Project/data"

# Creating a predictive model for IC50 based on molecular structure (SMILES)
Data from the ChEMBL database will now be compared against each compound's molecular fingerprint as described by the PaDEL database.

In [24]:
! cd "/content/drive/My Drive/ERK2 Project/data/"
file_path = "/content/drive/My Drive/ERK2 Project/data/ERK2_chembl_processed.csv"
processed = pd.read_csv(file_path)

In [25]:
smile_df = processed[['smiles', 'chembl_id']]
smile_df.to_csv('molecule.smi', sep = '\t', index=False, header=False)
! cp molecule.smi "/content/drive/My Drive/ERK2 Project/data"

In [26]:
! cat molecule.smi | wc -l

3846


###Retrieving PaDEL descriptor Data

Now that we have a file mapping SMILES strings to IC50, let's load the PaDEL descriptor software to convert our SMILES strings to the 801 fingerprint ids needed to build a robust regression model.

In [ ]:
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.sh
! unzip padel.zip

--2024-08-08 15:18:29--  https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip [following]
--2024-08-08 15:18:30--  https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25768637 (25M) [application/zip]
Saving to: ‘padel.zip’

padel.zip           100%[===================>]  24.57M  --.-KB/s    in 0.1s    

2024-08-08 15:18:30 (166 MB/s) - ‘padel.zip’ saved [25768637/25768637]

--2024-08-08 15:18:30--  https://github

In [ ]:
# descriptor of PaDEL standard configurations
! cat padel.sh

# converting descriptors for each compound interacting with ERK2
# note: this can take awhile with large molecule.smi files.
# Enjoy a cup of coffee while you wait!
! bash padel.sh

java -Xms1G -Xmx1G -Djava.awt.headless=true -jar ./PaDEL-Descriptor/PaDEL-Descriptor.jar -removesalt -standardizenitro -fingerprints -descriptortypes ./PaDEL-Descriptor/PubchemFingerprinter.xml -dir ./ -file descriptors_output.csv
Processing CHEMBL397591 in molecule.smi (1/3846). 
Processing CHEMBL35482 in molecule.smi (2/3846). 
Processing CHEMBL388978 in molecule.smi (3/3846). Average speed: 9.79 s/mol.
Processing CHEMBL356164 in molecule.smi (4/3846). Average speed: 5.29 s/mol.
Processing CHEMBL361708 in molecule.smi (5/3846). Average speed: 3.57 s/mol.
Processing CHEMBL255465 in molecule.smi (6/3846). Average speed: 2.77 s/mol.
Processing CHEMBL151430 in molecule.smi (7/3846). Average speed: 2.32 s/mol.
Processing CHEMBL357047 in molecule.smi (8/3846). Average speed: 2.02 s/mol.
Processing CHEMBL440356 in molecule.smi (9/3846). Average speed: 1.77 s/mol.
Processing CHEMBL359106 in molecule.smi (10/3846). Average speed: 1.55 s/mol.
Processing CHEMBL75680 in molecule.smi (11/3846). A

In [ ]:
padel_df = pd.read_csv('descriptors_output.csv')
padel_df.head()

#setting the inputs of the predictive model (PaDEL descriptors, or "X" input)
#here, the fingerprints will be set as the inputs of the predictive model,
#using a random forest ML algorithm to produce a function converting fingerprints
#to a prediction for pIC50 (instead of IC50 to avoid bias)
model_X = padel_df.drop(columns=['Name'])
model_X

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3841,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3842,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3843,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3844,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#setting Y values based on ChEMBL database
model_Y = processed['pIC50']

train_data = pd.concat([model_X, model_Y], axis=1)
train_data.to_csv('ERK2_pIC50_fp_train.csv')
! cp ERK2_pIC50_fp_train.csv "/content/drive/My Drive/ERK2 Project/data"

In [ ]:
train_data.head()

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,pIC50
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,5.602060
1,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,5.552842
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,8.602060
3,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,5.602060
4,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,8.045757


### Training data has been made!

In a separate script, the training data will be input into a random forest machine learning model to derive accurate predictions for pIC50 given SMILES data.